# Food Delivery Hackathon – Innomatics Entrance Test



## 1. Import Libraries


In [2]:
import pandas as pd
import sqlite3


## 2. Load Data Files


In [4]:
# Load orders CSV
orders = pd.read_csv("orders.csv")

# Load users JSON
users = pd.read_json("users.json")

# Load restaurants from SQL (reset DB to avoid conflicts)
import os

if os.path.exists("restaurants.db"):
    os.remove("restaurants.db")

conn = sqlite3.connect("restaurants.db")

with open("restaurants.sql") as f:
    conn.executescript(f.read())

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

# Preview data
orders.head(), users.head(), restaurants.head()



(   order_id  user_id  restaurant_id  order_date  total_amount  \
 0         1     2508            450  18-02-2023        842.97   
 1         2     2693            309  18-01-2023        546.68   
 2         3     2084            107  15-07-2023        163.93   
 3         4      319            224  04-10-2023       1155.97   
 4         5     1064            293  25-12-2023       1321.91   
 
                   restaurant_name  
 0               New Foods Chinese  
 1  Ruchi Curry House Multicuisine  
 2           Spice Kitchen Punjabi  
 3          Darbar Kitchen Non-Veg  
 4       Royal Eatery South Indian  ,
    user_id    name       city membership
 0        1  User_1    Chennai    Regular
 1        2  User_2       Pune       Gold
 2        3  User_3  Bangalore       Gold
 3        4  User_4  Bangalore    Regular
 4        5  User_5       Pune       Gold,
    restaurant_id restaurant_name  cuisine  rating
 0              1    Restaurant_1  Chinese     4.8
 1              2    Res

## 3. Merge the Datasets (Left Joins)


In [5]:
# Merge orders with users
merged1 = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)

# Merge with restaurants
final_df = pd.merge(
    merged1,
    restaurants,
    on="restaurant_id",
    how="left"
)

final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [6]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)



## 4. Load Final Dataset for Analysis


In [7]:
df = pd.read_csv("final_food_delivery_dataset.csv")
df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


## 5. MCQ Analysis


In [8]:
df[df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .sum() \
    .sort_values(ascending=False)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [9]:
df.groupby("cuisine")["total_amount"] \
    .mean() \
    .sort_values(ascending=False)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [10]:
(df.groupby("user_id")["total_amount"].sum() > 1000).sum()


np.int64(2544)

In [11]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

df["rating_range"] = pd.cut(df["rating"], bins=bins, labels=labels)

df.groupby("rating_range")["total_amount"] \
    .sum() \
    .sort_values(ascending=False)


/var/folders/81/mxqqsz7n1mj614t_q7t0wjyh0000gn/T/ipykernel_1887/1497673880.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"] \


rating_range
4.6–5.0    2197030.75
4.1–4.5    1960326.26
3.0–3.5    1881754.57
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [12]:
df[df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .mean() \
    .sort_values(ascending=False)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [13]:
rest_count = df.groupby("cuisine")["restaurant_id"].nunique()
rev = df.groupby("cuisine")["total_amount"].sum()

pd.concat([rest_count, rev], axis=1).rename(
    columns={
        "restaurant_id": "restaurant_count",
        "total_amount": "revenue"
    }
).sort_values("restaurant_count")


,restaurant_count,revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [14]:
(df["membership"].value_counts(normalize=True) * 100).round()


membership
Regular    50.0
Gold       50.0
Name: proportion, dtype: float64

In [15]:
df.groupby(
    ["membership", "cuisine"]
)["total_amount"].sum().sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [16]:
df["order_date"] = pd.to_datetime(df["order_date"], dayfirst=True)
df["quarter"] = df["order_date"].dt.quarter

df.groupby("quarter")["total_amount"] \
    .sum() \
    .sort_values(ascending=False)


quarter
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64

## 6. Numerical Answers


In [17]:
# Total orders by Gold members
total_gold_orders = (df["membership"] == "Gold").sum()

# Hyderabad total revenue (rounded)
hyd_revenue = round(df[df["city"] == "Hyderabad"]["total_amount"].sum())

# Distinct users
distinct_users = df["user_id"].nunique()

# Gold average order value
gold_avg_order = round(df[df["membership"] == "Gold"]["total_amount"].mean(), 2)

# Orders with rating >= 4.5
high_rating_orders = (df["rating"] >= 4.5).sum()

# Gold orders in Chennai
gold_chennai_orders = len(
    df[
        (df["membership"] == "Gold") &
        (df["city"] == "Chennai")
    ]
)

total_gold_orders, hyd_revenue, distinct_users, gold_avg_order, high_rating_orders, gold_chennai_orders


(np.int64(4987), 1889367, 2883, np.float64(797.15), np.int64(3374), 1337)

## 7. Fill-in-the-Blank Answers

- Join column between orders and users: user_id  
- Cuisine & rating dataset format: SQL  
- Total rows in final dataset: 9974  
- Missing users appear as: NaN  
- Pandas function used: merge()  
- Membership comes from: users.json  
- Restaurant join key: restaurant_id  
- Food type column: cuisine  
- Repeat orders cause user info to appear multiple times  
